In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from transformers import BertTokenizer, BertModel, BertConfig
from torch.nn.utils.rnn import pack_padded_sequence

In [3]:
# 특성 추출기 layer

# vgg_config = [64, 'N', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

def get_vgg_layer(config, batch_norm):
    layers = []
    in_channels = 3

    for c in config:
        assert c == 'M' or isinstance(c, int)
        
        if c == 'M': # c가 M이면 MaxPooling
            layers += [nn.MaxPool2d(kernel_size= 2)]

        else: # c가 int면 Convolution
            conv2d = nn.Conv2d(in_channels, c, kernel_size= 3, padding= 1)

            # batch normalization 적용
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace= True)]
            else:
                layers += [conv2d, nn.ReLU(inplace= True)]

            in_channels = c # 다음 layer의 in_channels로 사용

    return nn.Sequential(*layers)

In [ ]:
# 